# Pyspark: Recommender system

In [1]:
import findspark
findspark.init('/home/srabin/spark-3.5.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

24/08/07 18:09:15 WARN Utils: Your hostname, DESKTOP-P0KTK6U resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/07 18:09:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 18:09:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/07 18:09:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/07 18:09:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/08/07 18:09:16 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/08/07 18:09:16 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/08/07 18:09:16 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port

In [2]:
from pyspark.ml.recommendation import ALS

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
data = spark.read.csv('movielens_ratings.csv', inferSchema= True, header = True)

In [5]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
training, test = data.randomSplit([0.7, 0.3])

In [8]:
als = ALS(maxIter = 5, regParam = 0.01, userCol = 'userId', itemCol = 'movieId', ratingCol = 'rating')

In [9]:
model = als.fit(training)

24/08/07 18:14:37 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/08/07 18:14:37 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [10]:
predictions = model.transform(test)

In [11]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      0|   3.0|    28| -1.3189957|
|      2|   4.0|    28|  1.7508816|
|      3|   1.0|    26|-0.41795892|
|      2|   2.0|     1|    2.57038|
|      3|   1.0|     1| 0.28812385|
|      0|   1.0|    13|   1.478679|
|      0|   1.0|     6|  0.5492722|
|      2|   2.0|    20|   4.117182|
|      4|   1.0|     5| 0.54490626|
|      0|   1.0|    19|  1.1769027|
|      3|   1.0|     9| -1.7746772|
|      2|   1.0|    23| -5.4261417|
|      1|   1.0|     7|  1.6017394|
|      4|   1.0|     7| -0.7674225|
|      2|   4.0|    10|  0.2206416|
|      1|   3.0|    25|-0.26600462|
|      2|   1.0|    25|  2.6617644|
|      3|   1.0|    29|-0.41103297|
|      4|   1.0|    29|    4.22758|
|      0|   1.0|    11|-0.89375186|
+-------+------+------+-----------+
only showing top 20 rows



In [13]:
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol = 'rating', predictionCol = 'prediction') # root mean squared error = rmse

In [15]:
rmse = evaluator.evaluate(predictions)

In [16]:
print('RMSE:')
print(rmse)

RMSE:
2.0405832628082954


In [17]:
# what if we have a single user?
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])

In [19]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|      6|    11|
|     18|    11|
|     27|    11|
|     30|    11|
|     32|    11|
|     35|    11|
|     41|    11|
|     48|    11|
|     61|    11|
|     64|    11|
|     66|    11|
|     67|    11|
|     75|    11|
|     77|    11|
|     80|    11|
|     82|    11|
|     94|    11|
+-------+------+



In [21]:
recommendations = model.transform(single_user)

In [22]:
recommendations.orderBy('prediction', ascending = False).show()

+-------+------+------------+
|movieId|userId|  prediction|
+-------+------+------------+
|     64|    11|   3.6422646|
|     32|    11|   2.5151095|
|     27|    11|    1.983849|
|     48|    11|   1.8903351|
|     66|    11|   1.5343828|
|      6|    11|   1.5050746|
|     30|    11|   1.4067416|
|     94|    11|   0.8130052|
|     61|    11|   0.6076099|
|     82|    11|-0.056153223|
|     75|    11| -0.07139644|
|     18|    11| -0.19947168|
|     80|    11| -0.47043937|
|      0|    11| -0.89375186|
|     35|    11| -0.89503276|
|     41|    11|  -1.3396571|
|     67|    11|  -1.3458642|
|     77|    11|  -1.6785713|
+-------+------+------------+

